# **Week 3 Assignment**

## Part 1

In [ ]:
import pandas as pd
import numpy as np

*Reading the table 

In [3]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df.rename(columns={'Neighbourhood':'Neighborhood'}, inplace=True)
df.head(10)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


Discarding cells with Not assigned

In [4]:
df = df[df['Borough'] != 'Not assigned']
df.head(10)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


Pre-Processing the table

In [5]:
df_new = df.groupby('Postal Code', sort=False).agg(', '.join)
df_new.head(10)

,Borough,Neighborhood
Postal Code,,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Regent Park, Harbourfront"
M6A,North York,"Lawrence Manor, Lawrence Heights"
M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
M1B,Scarborough,"Malvern, Rouge"
M3B,North York,Don Mills
M4B,East York,"Parkview Hill, Woodbine Gardens"


In [6]:
df_new.loc[df_new['Neighborhood'] =='Not assigned', 'Neighborhood'] = df_new.loc[df_new['Neighborhood'] =='Not assigned', 'Borough']
df_new.reset_index(inplace=True)
df_new.head(10)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


Shape of dataset

In [7]:
df_new.shape

(103, 3)

## Part 2

In [8]:
!pip install geocoder
import geocoder
print('Done')

     |████████████████████████████████| 98 kB 7.2 MB/s eta 0:00:011
Done


Creating new columns for long. and latd.

In [9]:
df_new['Latitude'] = None
df_new['Longitude'] = None
df_new.head(5)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,None,None
1,M4A,North York,Victoria Village,None,None
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",None,None
3,M6A,North York,"Lawrence Manor, Lawrence Heights",None,None
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",None,None


Filling with the new cells with respective Long. and Latd. values

In [10]:
for i, pc in enumerate(df_new['Postal Code']):
    lat_lng_coords = None
    
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(pc))
        lat_lng_coords = g.latlng
    
    if lat_lng_coords:
        latitude = lat_lng_coords[0]
        longitude = lat_lng_coords[1]
    
    df_new.loc[i, 'Latitude'] = latitude
    df_new.loc[i, 'Longitude'] = longitude

df_new.head(10)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.7525,-79.3299
1,M4A,North York,Victoria Village,43.7306,-79.3131
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7233,-79.4504
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.6626,-79.5283
6,M1B,Scarborough,"Malvern, Rouge",43.8114,-79.1966
7,M3B,North York,Don Mills,43.7492,-79.3619
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.7072,-79.3119
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.6574,-79.378


## Part 3

Visualization

In [13]:
address = 'Toronto, CA'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [14]:
!pip install folium
import folium
print('Done')

     |████████████████████████████████| 93 kB 2.9 MB/s  eta 0:00:01
Done


Creating map of Toronto with the neighborhoods 

In [15]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Foursquare details

In [17]:
CLIENT_ID = 'XJ1O5BOMLLBDRR40P0SCLX5FWTQYJQ1OHO4UA0RCWXCGVKS4'
CLIENT_SECRET = 'ZMBM5Z41FVCLVOAZRVQC5QV1RH0RG1WOZYQMJPWEMG0YIJRQ'
VERSION = '20201210'
LIMIT = 100

Obtaining all nearby neighborhoods

In [20]:
import requests
LIMIT = 50
radius = 500

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [22]:
venues = getNearbyVenues(names=df_new['Neighborhood'], latitudes=df_new['Latitude'], longitudes=df_new['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

KeyError: 'groups'

In [23]:
print(venues.shape)
venues.head()

(1819, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
3,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store
4,Victoria Village,43.73057,-79.31306,Guardian Drug,43.730584,-79.307432,Pharmacy


In [24]:
venues.groupby('Neighborhood', as_index=False).count()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Agincourt,16,16,16,16,16,16
1,"Alderwood, Long Branch",4,4,4,4,4,4
2,Bayview Village,4,4,4,4,4,4
3,"Bedford Park, Lawrence Manor East",19,19,19,19,19,19
4,Berczy Park,50,50,50,50,50,50
...,...,...,...,...,...,...,...
91,"Willowdale, Willowdale West",5,5,5,5,5,5
92,Woburn,4,4,4,4,4,4
93,Woodbine Heights,18,18,18,18,18,18
94,York Mills West,3,3,3,3,3,3


# Analyzing Neighborhoods

In [25]:
# one hot encoding
venues_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

venues_onehot.head()

,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Processing dataset

In [26]:
venues_onehot['Neighborhood'] = venues['Neighborhood']
venues_onehot.head()

,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
temp = list(venues_onehot.columns)

if 'Neighborhood' in temp:
    temp.remove('Neighborhood')
    
fixed_columns = ['Neighborhood'] + temp
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Neighborhood,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
venues_grouped = venues_onehot.groupby('Neighborhood', sort=False).mean().reset_index()
print(venues_grouped.shape)
venues_grouped.head(10)

(96, 252)


,Neighborhood,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio,Zoo Exhibit
0,Parkwoods,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000
1,Victoria Village,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000
2,"Regent Park, Harbourfront",0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.05,0.000000
3,"Lawrence Manor, Lawrence Heights",0.04,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.04,0.00,0.000000
4,"Queen's Park, Ontario Provincial Government",0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000
5,"Islington Avenue, Humber Valley Village",0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000
6,"Malvern, Rouge",0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.333333
7,Don Mills,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000
8,"Parkview Hill, Woodbine Gardens",0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.066667,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000
9,"Garden District, Ryerson",0.00,0.0,0.0,0.02,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.00,0.000000


Top 3 categories

In [30]:
num_top_venues = 3

for hood in venues_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = venues_grouped[venues_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Parkwoods----
               venue  freq
0  Food & Drink Shop   0.5
1               Park   0.5
2             Office   0.0


----Victoria Village----
           venue  freq
0  Grocery Store  0.33
1       Pharmacy  0.33
2           Park  0.33


----Regent Park, Harbourfront----
            venue  freq
0     Coffee Shop  0.25
1  Breakfast Spot  0.10
2      Food Truck  0.05


----Lawrence Manor, Lawrence Heights----
                    venue  freq
0          Clothing Store  0.22
1  Furniture / Home Store  0.06
2     American Restaurant  0.04


----Queen's Park, Ontario Provincial Government----
            venue  freq
0     Coffee Shop  0.28
1  Sandwich Place  0.11
2       Gastropub  0.06


----Islington Avenue, Humber Valley Village----
      venue  freq
0  Pharmacy  0.25
1      Park  0.12
2      Café  0.12


----Malvern, Rouge----
                    venue  freq
0             Zoo Exhibit  0.33
1  Furniture / Home Store  0.33
2    Fast Food Restaurant  0.33


----Don Mills----
       

         venue  freq
0    Gift Shop  0.14
1  Pizza Place  0.14
2     Pharmacy  0.14


----Dorset Park, Wexford Heights, Scarborough Town Centre----
                       venue  freq
0                     Bakery   0.5
1                  Gift Shop   0.5
2  Middle Eastern Restaurant   0.0


----York Mills West----
               venue  freq
0          Speakeasy  0.33
1  Convenience Store  0.33
2               Park  0.33


----Davisville North----
               venue  freq
0              Hotel  0.12
1               Park  0.12
2  Food & Drink Shop  0.12


----Forest Hill North & West, Forest Hill Road Park----
                 venue  freq
0    French Restaurant   0.5
1                 Park   0.5
2  American Restaurant   0.0


----High Park, The Junction South----
                                      venue  freq
0  Residential Building (Apartment / Condo)  0.25
1                         Convenience Store  0.25
2                                      Park  0.25


----Westmount----
         

Creating dataframe with most common venues

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

top 5 venues of each neighborhood

In [32]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = venues_grouped['Neighborhood']

for ind in np.arange(venues_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(venues_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Parkwoods,Food & Drink Shop,Park,Eastern European Restaurant,Flea Market,Fish Market
1,Victoria Village,Park,Pharmacy,Grocery Store,Zoo Exhibit,Elementary School
2,"Regent Park, Harbourfront",Coffee Shop,Breakfast Spot,Pub,Spa,Food Truck
3,"Lawrence Manor, Lawrence Heights",Clothing Store,Furniture / Home Store,American Restaurant,Toy / Game Store,Men's Store
4,"Queen's Park, Ontario Provincial Government",Coffee Shop,Sandwich Place,Gastropub,Theater,Mediterranean Restaurant
...,...,...,...,...,...,...
91,"The Kingsway, Montgomery Road, Old Mill North",Pool,Zoo Exhibit,Eastern European Restaurant,Flea Market,Fish Market
92,Church and Wellesley,Coffee Shop,Gay Bar,Japanese Restaurant,Dance Studio,Restaurant
93,"Business reply mail Processing Centre, South C...",Coffee Shop,Restaurant,Hotel,Concert Hall,Sushi Restaurant
94,"Old Mill South, King's Mill Park, Sunnylea, Hu...",Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Bank


# Clustering

In [33]:
from sklearn.cluster import KMeans

In [34]:
k = 5
venues_grouped_clustering = venues_grouped.drop('Neighborhood', axis=1)
kmeans = KMeans(n_clusters=k, random_state=67).fit(venues_grouped_clustering)
kmeans.labels_

array([4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4,
       0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 0, 0, 0], dtype=int32)

In [35]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
venues_merged = df_new
venues_merged = venues_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
venues_merged.dropna(inplace=True)

The processed dataframe

In [36]:
print(venues_merged.shape)
venues_merged['Cluster Labels'] = venues_merged['Cluster Labels'].astype(int)
venues_merged

(100, 11)


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M3A,North York,Parkwoods,43.7525,-79.3299,4,Food & Drink Shop,Park,Eastern European Restaurant,Flea Market,Fish Market
1,M4A,North York,Victoria Village,43.7306,-79.3131,0,Park,Pharmacy,Grocery Store,Zoo Exhibit,Elementary School
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.6551,-79.3626,0,Coffee Shop,Breakfast Spot,Pub,Spa,Food Truck
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.7233,-79.4504,0,Clothing Store,Furniture / Home Store,American Restaurant,Toy / Game Store,Men's Store
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.6625,-79.3919,0,Coffee Shop,Sandwich Place,Gastropub,Theater,Mediterranean Restaurant
...,...,...,...,...,...,...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.6532,-79.5111,2,Pool,Zoo Exhibit,Eastern European Restaurant,Flea Market,Fish Market
99,M4Y,Downtown Toronto,Church and Wellesley,43.6666,-79.3813,0,Coffee Shop,Gay Bar,Japanese Restaurant,Dance Studio,Restaurant
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.6487,-79.3854,0,Coffee Shop,Restaurant,Hotel,Concert Hall,Sushi Restaurant
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.6328,-79.4894,0,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Bank


Visualizing cluster on map

In [37]:
import matplotlib.cm as cm
import matplotlib.colors as colors

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=12)

x = np.arange(k)
ys = [i + x + (i*x)**2 for i in range(k)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(venues_merged['Latitude'], venues_merged['Longitude'], venues_merged['Neighborhood'], venues_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Venues in each cluster

In [38]:
venues_merged.loc[venues_merged['Cluster Labels'] == 0, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
1,North York,0,Park,Pharmacy,Grocery Store,Zoo Exhibit,Elementary School
2,Downtown Toronto,0,Coffee Shop,Breakfast Spot,Pub,Spa,Food Truck
3,North York,0,Clothing Store,Furniture / Home Store,American Restaurant,Toy / Game Store,Men's Store
4,Downtown Toronto,0,Coffee Shop,Sandwich Place,Gastropub,Theater,Mediterranean Restaurant
5,Etobicoke,0,Pharmacy,Grocery Store,Park,Café,Skating Rink
...,...,...,...,...,...,...,...
97,Downtown Toronto,0,Café,Coffee Shop,Restaurant,Gym,Hotel
99,Downtown Toronto,0,Coffee Shop,Gay Bar,Japanese Restaurant,Dance Studio,Restaurant
100,East Toronto,0,Coffee Shop,Restaurant,Hotel,Concert Hall,Sushi Restaurant
101,Etobicoke,0,Chinese Restaurant,Italian Restaurant,Fast Food Restaurant,Sushi Restaurant,Bank


Viewing each cluster

In [39]:
venues_merged.loc[venues_merged['Cluster Labels'] == 1, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
71,Scarborough,1,Auto Garage,Zoo Exhibit,Electronics Store,Flower Shop,Flea Market


In [40]:
venues_merged.loc[venues_merged['Cluster Labels'] == 2, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
98,Etobicoke,2,Pool,Zoo Exhibit,Eastern European Restaurant,Flea Market,Fish Market


In [41]:
venues_merged.loc[venues_merged['Cluster Labels'] == 3, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
26,Scarborough,3,Trail,Zoo Exhibit,Electronics Store,Flower Shop,Flea Market


In [42]:
venues_merged.loc[venues_merged['Cluster Labels'] == 4, venues_merged.columns[[1] + list(range(5, venues_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,North York,4,Food & Drink Shop,Park,Eastern European Restaurant,Flea Market,Fish Market
27,North York,4,Park,Residential Building (Apartment / Condo),Zoo Exhibit,Eastern European Restaurant,Fish Market
45,North York,4,Park,Zoo Exhibit,Eastern European Restaurant,Flea Market,Fish Market
49,North York,4,Bakery,Park,Zoo Exhibit,Flower Shop,Flea Market
68,Central Toronto,4,French Restaurant,Park,Zoo Exhibit,Electronics Store,Flea Market
